In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(nn4_small2_pretrained, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
from IPython.display import Image
Image(filename='model_plot.png')

### Creating Facenet model

The CNN architecture used here is a variant of the inception architecture.

In [ ]:
from __future__ import division

In [ ]:
from model import create_model

nn4_small2 = create_model()


Model training aims to learn an embedding $f(x)$ of image $x$ such that the squared L2 distance between all faces of the same identity is small and the distance between a pair of faces from different identities is large. This can be achieved with a *triplet loss* $L$ that is minimized when the distance between an anchor image $x^a_i$ and a positive image $x^p_i$ (same identity) in embedding space is smaller than the distance between that anchor image and a negative image $x^n_i$ (different identity) by at least a margin $\alpha$.

$$L = \sum^{m}_{i=1} \large[ \small {\mid \mid f(x_{i}^{a}) - f(x_{i}^{p})) \mid \mid_2^2} - {\mid \mid f(x_{i}^{a}) - f(x_{i}^{n})) \mid \mid_2^2} + \alpha \large ] \small_+$$



In [ ]:
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

### Face alignment

In [ ]:
import numpy as np
import os.path

class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
s='jpg'
a="jpeg"    
b="JPG" 
c="png" 
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        #print(i)
        for f in os.listdir(os.path.join(path, i)):
             if f.endswith(s) or f.endswith(a)  or f.endswith(b) or f.endswith(c):
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('data')
print(metadata)
print(metadata.shape[0])

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from align import AlignDlib

%matplotlib inline

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    #destRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img[...,::-1]
    #return destRGB

# Initialize the OpenFace face alignment utility
alignment = AlignDlib('models/landmarks.dat')

# Load an image 
jc_orig = load_image(metadata[55].image_path())

# Detect face and return bounding box
bb = alignment.getLargestFaceBoundingBox(jc_orig)

# Transform image using specified face landmark indices and crop image to 96x96
jc_aligned = alignment.align(96, jc_orig, bb, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

# Show original image
f = plt.figure(figsize=(20,20))
f.add_subplot(131)

#plt.subplots(figsize=(4, 4))
plt.imshow(jc_orig)
#f.savefig("example.jpg")
# Show original image with bounding box
f.add_subplot(132)
#subplots(figsize=(4, 4))
plt.imshow(jc_orig)
plt.gca().add_patch(patches.Rectangle((bb.left(), bb.top()), bb.width(), bb.height(), fill=False, color='red'))
#f.savefig("detect_face.jpg")
# Show aligned image
f.add_subplot(133)
#plt.subplots(figsize=(4, 4))
plt.imshow(jc_aligned);
#f.savefig("align_face.jpg")

In [ ]:
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

### Embedding vectors

Embedding vectors can now be calculated by feeding the aligned and scaled images into the pre-trained network.

In [ ]:
import numpy as np
embedded = np.zeros((metadata.shape[0], 128))

#print(metadata)
k=0
for i, m in enumerate(metadata):
    
        print(i,m.image_path())
        img = load_image(m.image_path())
        
        img = align_image(img)
        # scale RGB values to interval [0,1]
        if img is not None:
            img = (img / 255.).astype(np.float32)
        # obtain embedding vector for image
            embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
      

Let's verify on a single triplet example that the squared L2 distance between its anchor-positive pair is smaller than the distance between its anchor-negative pair.

In [ ]:
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

def show_pair(idx1, idx2):
    f=plt.figure(figsize=(10,5))
    plt.suptitle(f'Distance = {distance(embedded[idx1], embedded[idx2]):.2f}')
    plt.subplot(121)
    
    plt.imshow(load_image(metadata[idx1].image_path()))
    plt.subplot(122)
    plt.imshow(load_image(metadata[idx2].image_path())); 
 
show_pair(0, 8)
show_pair(0, 126)


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

distances = [] # squared L2 distance between pairs
identical = [] # 1 if same identity, 0 otherwise

num = len(metadata)

for i in range(num - 1):
    for j in range(1, num):
        distances.append(distance(embedded[i], embedded[j]))
        identical.append(1 if metadata[i].name == metadata[j].name else 0)
        
distances = np.array(distances)
identical = np.array(identical)

thresholds = np.arange(0.3, 1.0, 0.01)

f1_scores = [f1_score(identical, distances < t) for t in thresholds]
acc_scores = [accuracy_score(identical, distances < t) for t in thresholds]

opt_idx = np.argmax(f1_scores)
# Threshold at maximal F1 score
opt_tau = thresholds[opt_idx]
# Accuracy at maximal F1 score
opt_acc = accuracy_score(identical, distances < opt_tau)

# Plot F1 score and accuracy as function of distance threshold
plt.plot(thresholds, f1_scores, label='F1 score');
plt.plot(thresholds, acc_scores, label='Accuracy');
plt.axvline(x=opt_tau, linestyle='--', lw=1, c='lightgrey', label='Threshold')
plt.title(f'Accuracy at threshold {opt_tau:.2f} = {opt_acc:.3f}');
plt.xlabel('Distance threshold')
plt.legend();

### Distance threshold

In [ ]:
dist_pos = distances[identical == 1]
dist_neg = distances[identical == 0]

fig=plt.figure(figsize=(15,5))

plt.subplot(121)
plt.hist(dist_pos)
plt.axvline(x=opt_tau, linestyle='--', lw=1, c='lightgrey', label='Threshold')
plt.title('Distances (pos. pairs)')
plt.legend();

plt.subplot(122)
plt.hist(dist_neg)
plt.axvline(x=opt_tau, linestyle='--', lw=1, c='lightgrey', label='Threshold')
plt.title('Distances (neg. pairs)')
plt.legend();
fig.savefig("dist_histo.jpg")

### Classifier 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

targets = np.array([m.name for m in metadata])

encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

train_idx = np.arange(metadata.shape[0]) % 2 != 0
test_idx = np.arange(metadata.shape[0]) % 2 == 0


# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

knn = KNeighborsClassifier(n_neighbors=1,metric='euclidean')
svc = SVC(kernel='linear', probability=True)

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

acc_knn = accuracy_score(y_test, knn.predict(X_test))
acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'KNN accuracy = {acc_knn}, SVM accuracy = {acc_svc}')

### training with all  traininig data


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

targets = np.array([m.name for m in metadata])

#print(targets)
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

train_idx = np.arange(metadata.shape[0]) 

#test_idx = np.arange(metadata.shape[0]) % 2 == 0


# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
#X_test = embedded[test_idx]

y_train = y[train_idx]
#y_test = y[test_idx]

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
svc = SVC(kernel='linear', probability=True)

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

#acc_knn = accuracy_score(y_test, knn.predict(X_test))
#acc_svc = accuracy_score(y_test, svc.predict(X_test))

#print(f'KNN accuracy = {acc_knn}, SVM accuracy = {acc_svc}')

## Save Model

In [ ]:
import pickle
knn_model_save_path = "saved_models//knn_model.sav"
if knn_model_save_path != "":
    with open(knn_model_save_path, 'wb') as f:
        pickle.dump(knn, f)
        
svc_model_save_path = "saved_models//svc_model.sav"
if svc_model_save_path != "":
    with open(svc_model_save_path, 'wb') as f:
        pickle.dump(svc, f)
        
label_model_save_path = "saved_models//label_model.sav"
if label_model_save_path != "":
    with open(label_model_save_path, 'wb') as f:
        pickle.dump(encoder, f)

### Dataset visualization

To embed the dataset into 2D space for displaying identity clusters, [t-distributed Stochastic Neighbor Embedding](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) (t-SNE) is applied to the 128-dimensional embedding vectors. Except from a few outliers, identity clusters are well separated.

In [ ]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(embedded)
#print(X_embedded)
fig=plt.figure(figsize=(15,8))
for i, t in enumerate(set(targets)):
    idx = targets == t
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], label=t)   

plt.legend(bbox_to_anchor=(1, 1));
fig.savefig("cluster.jpg")

In [ ]:
embedded

# Save embedding vectors

In [ ]:
import csv
myFile = open('embedding.csv', 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(embedded)

# Save model as pb

In [ ]:
from model import create_model
import tensorflow as tf
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
from keras import backend as K

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in nn4_small2_pretrained.outputs])

In [ ]:
tf.train.write_graph(frozen_graph, "pb_file", "my_model.pb", as_text=False)

## Load Model  -  Independent

In [ ]:
import pickle
knn_model_save_path = "saved_models//knn_model.sav"
svc_model_save_path = "saved_models//svc_model.sav"
label_model_save_path = "saved_models//label_model.sav"
with open(knn_model_save_path, 'rb') as f:
    knn = pickle.load(f)
with open(svc_model_save_path, 'rb') as f:
    svc = pickle.load(f)
with open(label_model_save_path, 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
from model import create_model
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

# Webcam

In [ ]:
import cv2
import matplotlib.pyplot as plt
from align import AlignDlib
import numpy as np
alignment = AlignDlib('models/landmarks.dat')
camera = cv2.VideoCapture(0)
while True:
    return_value,image = camera.read()
    frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow('image',image)
    if cv2.waitKey(1)& 0xFF == ord('r'):
        bb = alignment.getAllFaceBoundingBoxes(frame)
        im_aligned=[]
        for i in bb:
              j = alignment.align(96, frame, i, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
              im_aligned.append(j)   
        for n in im_aligned:
            n = (n / 255.).astype(np.float32)
            embedded_t = nn4_small2_pretrained.predict(np.expand_dims(n, axis=0))[0]
            #print(embedded_t)
            embedded_t=embedded_t.reshape(1,-1)
            #print( embedded_t)
            example_predic = knn.predict(embedded_t)
            example_prob = svc.predict_proba(embedded_t)
            print(f"Probablities of each class is {example_prob}")
            if np.any(example_prob>0.15):
                example_i = encoder.inverse_transform(example_predic)[0]
                print(f'Recognized as {example_i}')
            else:
                print("Not a face from database...")
    if cv2.waitKey(1)& 0xFF == ord('q'):
        break
camera.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
from align import AlignDlib
import numpy as np
import tensorflow as tf
%matplotlib inline
# Initialize the OpenFace face alignment utility

start_time = time.clock()
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    #destRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img[...,::-1]
    #return destRGB

with tf.Session(graph=tf.Graph()) as sess:
    alignment = AlignDlib('models//landmarks.dat')

    # Load an image of Jacques Chirac
    im_orig = load_image('test_images//t6.jpg')

    # Detect face and return bounding box
    bb = alignment.getAllFaceBoundingBoxes(im_orig)
    #print(bb)
    # Transform image using specified face landmark indices and crop image to 96x96
    im_aligned=[]
    for i in bb:
          j = alignment.align(96, im_orig, i, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
          im_aligned.append(j)   

    image_height_tensor = tf.placeholder(tf.int32)
    image_width_tensor = tf.placeholder(tf.int32)

    # Show original image
    #plt.subplot(221)
    #plt.imshow(im_orig)
    export_path = "inference_graph//V11"
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    fig=plt.figure(figsize=(15,15))
    plt.imshow(im_orig)
    # Show original image with bounding box
    #plt.subplot(222)
    #print(im_aligned)
    #plt.gca().add_patch(patches.Rectangle((bb.left(), bb.top()), bb.width(), bb.height(), fill=False, color='red'))
    for i in bb:
        plt.gca().add_patch(patches.Rectangle((i.left(), i.top()), i.width(), i.height(), fill=False, color='red',linewidth=2))

    # Show aligned image
    #plt.subplot(133)
    #fig.savefig('time_hog_detection.jpg')
    #print("--- %s seconds ---" % (time.clock() - start_time))
    #for n in im_aligned:
    n = (n / 255.).astype(np.float32)
    print("n is")
    #print(n.shape)
    new_n = np.expand_dims(n, axis=0)
    #print("new_n is")
   # print(new_n.shape)
    tensor_info_input = tf.saved_model.utils.build_tensor_info(tf.convert_to_tensor(n))
    tensor_info_height = tf.saved_model.utils.build_tensor_info(image_height_tensor)
    tensor_info_width = tf.saved_model.utils.build_tensor_info(image_width_tensor)

    embedded_t = nn4_small2_pretrained.predict(new_n)[0]
    print(embedded_t.shape)

    #print(embedded)
    embedded_t=embedded_t.reshape(1,-1)
    print(embedded_t.shape)
    print( embedded_t)
    tensor_info_output = tf.saved_model.utils.build_tensor_info(tf.convert_to_tensor(embedded_t))
    example_predic = knn.predict(embedded_t)
    example_prob = svc.predict_proba(embedded_t)
    #print(example_prob)
    if np.any(example_prob>0.50):
        example_i = encoder.inverse_transform(example_predic)[0]
        plt.subplots(figsize=(4, 4))
        plt.imshow(n)

        plt.title(f'Recognized as {example_i}')

    else:
            plt.subplots(figsize=(4, 4))
            plt.imshow(n)

            plt.title('N/A')
    prediction_signature = (
            tf.saved_model.signature_def_utils.build_signature_def(
                inputs={'images': tensor_info_input, 'height': tensor_info_height, 'width': tensor_info_width},
                outputs={'segmentation_map': tensor_info_output},
                method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))
    builder.add_meta_graph_and_variables(
            sess, [tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                'predict_images':
                    prediction_signature,
            })

    # export the model
    builder.save(as_text=True)
    print('Done exporting!')
    print("--- %s seconds ---" % (time.clock() - start_time))

In [ ]:
import tensorflow as tf

def printTensors(pb_file):

    # read pb into graph_def
    with tf.gfile.GFile(pb_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # import graph_def
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    # print operations
    for op in graph.get_operations():
        print(op.name)


printTensors("pb_file/my_model.pb")

In [ ]:
import tensorflow as tf
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants

export_dir = 'inference_graph/V26'
graph_pb = 'pb_file/my_model.pb'

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

with tf.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

sigs = {}

with tf.Session(graph=tf.Graph()) as sess:
    # name="" is important to ensure we don't get spurious prefixing
    tf.import_graph_def(graph_def, name="")
    g = tf.get_default_graph()
    inp = g.get_tensor_by_name("conv1/kernel:0")
    out = g.get_tensor_by_name("norm_layer_2/l2_normalize/Sum/reduction_indices:0")

    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.saved_model.signature_def_utils.predict_signature_def(
            {"in": inp}, {"out": out})

    builder.add_meta_graph_and_variables(sess,
                                         [tag_constants.SERVING],
                                         signature_def_map=sigs)

builder.save()

# Correct pbtxt

In [ ]:
import keras.backend as K
import tensorflow as tf
from keras.applications import VGG19
from keras.models import Model
from model import create_model
# very important to do this as a first thing
K.set_learning_phase(0)
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [ ]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def, signature_constants
from tensorflow.contrib.session_bundle import exporter

export_path = 'inference_graph/V11'
builder = saved_model_builder.SavedModelBuilder(export_path)
prediction_signature = (tf.saved_model.signature_def_utils.build_signature_def(
            inputs={'images':tf.saved_model.utils.build_tensor_info(nn4_small2_pretrained.input)},
            outputs={'scores':tf.saved_model.utils.build_tensor_info(nn4_small2_pretrained.output)},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)) 

#signature = predict_signature_def(inputs={'images': nn4_small2_pretrained.input},outputs={'scores': nn4_small2_pretrained.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': prediction_signature})
    builder.save(as_text=True)

# Visualization and Explainability

In [ ]:
from numpy import genfromtxt
import numpy as np
embedded = genfromtxt('embedding.csv', delimiter=',')

In [ ]:
import os.path

class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
s='jpg'
a="jpeg"    
b="JPG" 
c="png" 
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        #print(i)
        for f in os.listdir(os.path.join(path, i)):
             if f.endswith(s) or f.endswith(a)  or f.endswith(b) or f.endswith(c):
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('data')
#print(metadata)
print(metadata.shape[0])
targets = np.array([m.name for m in metadata])

In [ ]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(embedded)
#print(X_embedded)
fig=plt.figure(figsize=(15,8))
for i, t in enumerate(set(targets)):
    idx = targets == t
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1],label=t)   
    
plt.legend(bbox_to_anchor=(1, 1));
fig.savefig("cluster.jpg")

In [ ]:
inception_layers = [ layer.name for layer in nn4_small2_pretrained.layers]
print(inception_layers)

In [ ]:
from keras.models import Model
activation_9_extractor = Model(inputs=nn4_small2_pretrained.input, outputs=nn4_small2_pretrained.get_layer('activation_39').output)

In [ ]:
activation_9_featres = activation_9_extractor.predict(np.expand_dims(n, axis=0))
print(activation_9_featres.shape)

In [ ]:
plt.imshow(activation_9_featres[0, :, :, 0], cmap='jet')

In [ ]:
def plot_feature_maps(feature_maps):
    height, width, depth = feature_maps.shape
    nb_plot = int(np.rint(np.sqrt(depth)))
    fig = plt.figure(figsize=(20, 20))
    for i in range(depth):
        plt.subplot(nb_plot, nb_plot, i+1)
        plt.imshow(feature_maps[:,:,i], cmap='jet')
        plt.title('feature map {}'.format(i+1))
    plt.show()

In [ ]:
def plot_layer(layer_name):    
    features_extractor = Model(inputs=nn4_small2_pretrained.input, outputs=nn4_small2_pretrained.get_layer(layer_name).output)
    feature_maps = features_extractor.predict(np.expand_dims(n, axis=0))[0]
    print("At layer \"{}\" : {} ".format(layer_name, feature_maps.shape))
    plot_feature_maps(feature_maps)

# Layers

In [ ]:
plot_layer('conv1')

In [ ]:
plot_layer('conv2')

In [ ]:
plot_layer('conv3')

In [ ]:
plot_layer('concatenate_1')

In [ ]:
plot_layer('concatenate_2')

In [ ]:
plot_layer('concatenate_3')

In [ ]:
plot_layer('concatenate_4')

In [ ]:
plot_layer('concatenate_5')

In [ ]:
plot_layer('concatenate_6')

In [ ]:
plot_layer('concatenate_7')

# DUMP

In [ ]:
%matplotlib notebook

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
from align import AlignDlib
import numpy as np


# Initialize the OpenFace face alignment utility

start_time = time.clock()
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    #destRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img[...,::-1]
    #return destRGB

alignment = AlignDlib('models/landmarks.dat')



vc = cv2.VideoCapture(0)

plt.ion()

if vc.isOpened(): # try to get the first frame
    is_capturing, frame = vc.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    webcam_preview = plt.imshow(frame)    
else:
    is_capturing = False

while is_capturing:
    try:    # Lookout for a keyboardInterrupt to stop the script
        is_capturing, frame = vc.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
        
        ##Detection here
        # Detect face and return bounding box
        bb = alignment.getAllFaceBoundingBoxes(frame)
        im_aligned=[]
        for i in bb:
              j = alignment.align(96, frame, i, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
              im_aligned.append(j)   
        for i in bb:
            plt.gca().add_patch(patches.Rectangle((i.left(), i.top()), i.width(), i.height(), fill=False, color='red',linewidth=2))
        for n in im_aligned:
            n = (n / 255.).astype(np.float32)
            embedded_t = nn4_small2_pretrained.predict(np.expand_dims(n, axis=0))[0]
            #print(embedded)
            embedded_t=embedded_t.reshape(1,-1)
            #print( embedded_t)
            example_predic = knn.predict(embedded_t)
            example_prob = svc.predict_proba(embedded_t)
            #print(example_prob)
            if np.any(example_prob>0.25):
                example_i = encoder.inverse_transform(example_predic)[0]
                print(f'Recognized as {example_i}')
           
            else:
                print('N/A')
        webcam_preview.set_data(frame)
        plt.draw()
        try:    # Avoids a NotImplementedError caused by `plt.pause`
            plt.pause(0.05)
        except Exception:
            pass
    except KeyboardInterrupt:
        vc.release()

In [ ]:
#Try one
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
from align import AlignDlib
import numpy as np
%matplotlib inline
# Initialize the OpenFace face alignment utility

start_time = time.clock()
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    #destRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img[...,::-1]
    #return destRGB

alignment = AlignDlib('models/landmarks.dat')

# Load an image of Jacques Chirac
im_orig = load_image('test_images//t6.jpg')

# Detect face and return bounding box
bb = alignment.getAllFaceBoundingBoxes(im_orig)
#print(bb)
# Transform image using specified face landmark indices and crop image to 96x96
im_aligned=[]
for i in bb:
      j = alignment.align(96, im_orig, i, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
      im_aligned.append(j)   

# Show original image
#plt.subplot(221)
#plt.imshow(im_orig)

fig=plt.figure(figsize=(15,15))
plt.imshow(im_orig)
# Show original image with bounding box
#plt.subplot(222)
#print(im_aligned)
#plt.gca().add_patch(patches.Rectangle((bb.left(), bb.top()), bb.width(), bb.height(), fill=False, color='red'))
for i in bb:
    
    plt.gca().add_patch(patches.Rectangle((i.left(), i.top()), i.width(), i.height(), fill=False, color='red',linewidth=2))

# Show aligned image
#plt.subplot(133)
#fig.savefig('time_hog_detection.jpg')
#print("--- %s seconds ---" % (time.clock() - start_time))
for n in im_aligned:
    n = (n / 255.).astype(np.float32)
    embedded_t = nn4_small2_pretrained.predict(np.expand_dims(n, axis=0))[0]
    print(embedded_t)
    print("Reshaping")
    embedded_t=embedded_t.reshape(1,-1)
    print( embedded_t)

    example_predic = knn.predict(embedded_t)
    example_prob = svc.predict_proba(embedded_t)
    #print(example_prob)
    if np.any(example_prob>0.25):
        example_i = encoder.inverse_transform(example_predic)[0]
        plt.subplots(figsize=(4, 4))
        plt.imshow(n)
    
        plt.title(f'Recognized as {example_i}')
           
    else:
            plt.subplots(figsize=(4, 4))
            plt.imshow(n)

            plt.title('N/A')
print("--- %s seconds ---" % (time.clock() - start_time))
            

In [ ]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk

width, height = 800, 600
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

root = tk.Tk()
root.bind('<Escape>', lambda e: root.quit())
lmain = tk.Label(root)
lmain.pack()

def show_frame():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    bb = alignment.getAllFaceBoundingBoxes(cv2image)
    im_aligned=[]
    for i in bb:
          j = alignment.align(96, frame, i, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
          im_aligned.append(j)   
    for i in bb:
        plt.gca().add_patch(patches.Rectangle((i.left(), i.top()), i.width(), i.height(), fill=False, color='red',linewidth=2))
    for n in im_aligned:
        n = (n / 255.).astype(np.float32)
        embedded_t = nn4_small2_pretrained.predict(np.expand_dims(n, axis=0))[0]
        #print(embedded)
        embedded_t=embedded_t.reshape(1,-1)
        #print( embedded_t)
        example_predic = knn.predict(embedded_t)
        example_prob = svc.predict_proba(embedded_t)
        #print(example_prob)
        if np.any(example_prob>0.45):
            example_i = encoder.inverse_transform(example_predic)[0]
            print(f'Recognized as {example_i}')

        else:
            print('N/A')
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, show_frame)

show_frame()
root.mainloop()